In [1]:
import os
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("chemical_loading") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

22/09/05 16:24:36 WARN Utils: Your hostname, Aymans-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.101 instead (on interface en0)
22/09/05 16:24:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/05 16:24:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
diapole_moments = spark.read.option("header",True).csv('../data/dipole_moments.csv')
magnet_sheilding_tensors = spark.read.option("header",True).csv('../data/magnetic_shielding_tensors.csv')
mulliken_charges = spark.read.option("header",True).csv('../data/mulliken_charges.csv')
potential_energy = spark.read.option("header",True).csv('../data/potential_energy.csv')
scalar_coupling_contribution = spark.read.option("header",True).csv('../data/scalar_coupling_contributions.csv')
structures = spark.read.option("header",True).csv('../data/structures.csv').withColumnRenamed("x","small_x") \
    .withColumnRenamed("y","small_y")\
    .withColumnRenamed("z","small_z")

In [3]:
dm_pe = diapole_moments.alias('a')\
        .join(potential_energy.alias('b'), diapole_moments.molecule_name == potential_energy.molecule_name, "left")\
        .select("a.molecule_name", "a.X", "a.Y", "a.Z", "b.potential_energy")

In [4]:
dm_pe_mc = dm_pe.alias('a')\
    .join(mulliken_charges.alias('b'), dm_pe.molecule_name == mulliken_charges.molecule_name, "left")\
    .select("a.molecule_name", "a.X", "a.Y", "a.Z", "a.potential_energy", "b.atom_index", "b.mulliken_charge")

In [5]:
dm_pe_mc_mst = dm_pe_mc.alias('a')\
    .join(
        magnet_sheilding_tensors.alias('b'), 
        (dm_pe_mc.molecule_name == magnet_sheilding_tensors.molecule_name) & (dm_pe_mc.atom_index == magnet_sheilding_tensors.atom_index)
        , "left")\
    .select(
        "a.molecule_name", 
        "a.X", 
        "a.Y",
        "a.Z", 
        "a.potential_energy", 
        "a.atom_index", 
        "a.mulliken_charge", 
        "b.XX", "b.YX", "b.ZX", "b.XY", "b.YY", "b.ZY", "b.XZ", "b.YZ", "b.ZZ")

In [6]:
dm_pe_mc_mst_scc = dm_pe_mc_mst.alias('a')\
    .join(
        scalar_coupling_contribution.alias('b'), 
        (dm_pe_mc_mst.molecule_name == scalar_coupling_contribution.molecule_name) & (dm_pe_mc_mst.atom_index == scalar_coupling_contribution.atom_index_0)
        , "left")\
    .select(
        "a.molecule_name",
        "a.X",
        "a.Y",
        "a.Z",
        "a.potential_energy",
        "a.atom_index",
        "a.mulliken_charge",
        "a.XX",
        "a.YX",
        "a.ZX",
        "a.XY",
        "a.YY",
        "a.ZY",
        "a.XZ",
        "a.YZ",
        "a.ZZ",
        "b.atom_index_0",
        "b.atom_index_1",
        "b.type",
        "b.fc",
        "b.sd",
        "b.pso",
        "b.dso"
)

In [7]:
dm_pe_mc_mst_scc_struc = dm_pe_mc_mst_scc.alias('a')\
    .join(
        structures.alias('b'), 
        (dm_pe_mc_mst_scc.molecule_name == structures.molecule_name) & (dm_pe_mc_mst.atom_index == structures.atom_index)
        , "left")\
    .select(
    "a.molecule_name",
    "a.X",
    "a.Y",
    "a.Z",
    "a.potential_energy",
    "a.atom_index",
    "a.mulliken_charge",
    "a.XX",
    "a.YX",
    "a.ZX",
    "a.XY",
    "a.YY",
    "a.ZY",
    "a.XZ",
    "a.YZ",
    "a.ZZ",
    "a.atom_index_0",
    "a.atom_index_1",
    "a.type",
    "a.fc",
    "a.sd",
    "a.pso",
    "a.dso",
    "b.atom",
    "b.small_x",
    "b.small_y",
    "b.small_z"
)

In [8]:
dm_pe_mc_mst_scc_struc.printSchema()

root
 |-- molecule_name: string (nullable = true)
 |-- X: string (nullable = true)
 |-- Y: string (nullable = true)
 |-- Z: string (nullable = true)
 |-- potential_energy: string (nullable = true)
 |-- atom_index: string (nullable = true)
 |-- mulliken_charge: string (nullable = true)
 |-- XX: string (nullable = true)
 |-- YX: string (nullable = true)
 |-- ZX: string (nullable = true)
 |-- XY: string (nullable = true)
 |-- YY: string (nullable = true)
 |-- ZY: string (nullable = true)
 |-- XZ: string (nullable = true)
 |-- YZ: string (nullable = true)
 |-- ZZ: string (nullable = true)
 |-- atom_index_0: string (nullable = true)
 |-- atom_index_1: string (nullable = true)
 |-- type: string (nullable = true)
 |-- fc: string (nullable = true)
 |-- sd: string (nullable = true)
 |-- pso: string (nullable = true)
 |-- dso: string (nullable = true)
 |-- atom: string (nullable = true)
 |-- small_x: string (nullable = true)
 |-- small_y: string (nullable = true)
 |-- small_z: string (nullable =

In [9]:
# dm_pe_mc_mst_scc_struc.write.partitionBy("molecule_name").option("header", True).parquet("../data/all_data_parquet")

In [10]:
dm_pe_mc_mst_scc_struc.write.option("header", True).csv("../data/all_data_csv")

22/09/05 16:24:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
